In [1]:
import pandas as pd
import time
import datetime
import csv
pd.set_option('mode.chained_assignment',  None)

In [2]:
# Read Combined Dataset downloaded from the github
epl = pd.read_csv("/Users/bradyhong/Downloads/SoccerPredictions-afe206f8403d87d12fa83bcb62d708820034eac7/Data/Combined_Data.csv")

In [3]:
# Get the all the possible teams
epl_team = set()
epl.home_team
for i in epl.home_team:
    epl_team.add(i)

In [4]:
# Making features and aggregate them to the combined dataset
def recency(depth):
    for a in range(depth):
        # Making features
        epl["t-"+str(a+1)+" home_attacking"] = 0
        epl["t-"+str(a+1)+" home_defending"] = 0
        epl["t-"+str(a+1)+" home_home"] = 0
        epl["t-"+str(a+1)+" home_strength"] = 0
        epl["home_strength"] = 0
        epl["t-"+str(a+1)+" away_home"] = 0
        epl["t-"+str(a+1)+" away_attacking"] = 0
        epl["t-"+str(a+1)+" away_defending"] = 0
        epl["t-"+str(a+1)+" away_strength"] = 0
        epl["away_strength"] = 0
    for i in epl_team:
        home = pd.DataFrame(epl[epl.home_team == i])
        away = pd.DataFrame(epl[epl.away_team == i])
        new_data = pd.concat([home, away]).sort_index() # Dataset for each team i
        index = new_data.index 
        for j in range(len(index)-depth):
            if new_data.loc[index[j+depth], "home_team"] == i:
                count = 0
                for c in range(depth):
                    past = new_data.loc[index[j+depth-c-1]]
                    if i == past.home_team:
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_attacking"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_defending"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_home"] = 1
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_strength"] = past.home_strength
                        count = count + int(past.home_score) - int(past.away_score)
                    else:
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_attacking"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_defending"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_home"] = -1
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_strength"] = past.away_strength
                        count = count + int(past.away_score) - int(past.home_score)
                new_data.loc[index[j+depth], "home_strength"] = count/depth
            else:
                count = 0
                for d in range(depth):
                    past = new_data.loc[index[j+depth-d-1]]
                    if i == past.home_team:
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_attacking"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_defending"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_home"] = 1
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_strength"] = past.home_strength
                        count = count + int(past.home_score) - int(past.away_score)
                    else:
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_attacking"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_defending"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_home"] = -1
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_strength"] = past.away_strength
                        count = count + int(past.away_score) - int(past.home_score)
                new_data.loc[index[j+depth], "away_strength"] = count/depth
        for x in index:
            epl.loc[x] = new_data.loc[x]

In [5]:
recency(depth = 3) # Run the recency feature with depth = 3 

In [6]:
epl.to_csv("recency.csv")

In [7]:
# Just to extract recency features only for depth 3
extract = epl[['home_team','away_team','home_score','away_score','home_strength','away_strength',
               't-3 home_attacking', 't-3 home_defending','t-3 home_strength', 't-3 home_home',
               't-3 away_attacking','t-3 away_defending','t-3 away_strength', 't-3 away_home',
               't-2 home_attacking','t-2 home_defending','t-2 home_strength','t-2 home_home',
               't-2 away_attacking', 't-2 away_defending','t-2 away_strength', 't-2 away_home',
               't-1 home_attacking', 't-1 home_defending','t-1 home_strength', 't-1 home_home',
               't-1 away_attacking', 't-1 away_defending','t-1 away_strength','t-1 away_home']]

In [8]:
extract.to_csv("extract.csv")

In [9]:
MU = pd.DataFrame(extract[extract.home_team == 'Manchester United'])

In [10]:
MU2 = pd.DataFrame(extract[extract.away_team == 'Manchester United'])

In [11]:
MU_data = pd.concat([MU, MU2]).sort_index() # Just make the data for Manchester United

In [12]:
MU_data.to_csv("mu.csv")